# EV State-Space Model

This  notebook is used to reproduce the State Space Model of aggregateed EV for frequency regulation.

Running on Jinning's local machine, in the env "ev".

Working notes:

02/06/2022: the EV data generation is completed. It looks weired, we may need update it later on.


In [2]:
import andes
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import logging
logger = logging.getLogger(__name__)

print(f"ANDES Version: {andes.__version__}")

ANDES Version: 1.6.5.post25.dev0+g9de1f559


## EV data

### Define parameters

The EV parameters are defined as two types. Type I follows uniform distribution, which is stored in a Dict `ev_param`. Type II follows normal distribution, which is stored in a pd.DataFrame `ev_pdf`.

The data are cited from:

M. Wang et al., "State Space Model of Aggregated Electric Vehicles for Frequency Regulation," in IEEE Transactions on Smart Grid, vol. 11, no. 2, pp. 981-994, March 2020, doi: 10.1109/TSG.2019.2929052.

In [43]:
class ev_ssm():
    """
    EV State Space Model.

    EV parameters:
    P, n, Q follow uniform distribution.

    Attributes
    ----------
    ev_data: pandas.DataFrame
        EV dataset
    N: int
        Number of EVs
    soc_intv: dict
        SoC interval
    ev_ufparam: dict
        EV uniform parameters range
    ev_nfparam: dict
        EV normal parameters range

    Notes
    -----
    ev_param
        Ns: Number of SoC intervals
        Pl: rated charging/discharging power (kW) lower bound
        Pu: rated charging/discharging power (kW) upper bound
        nl: charging/discharging efficiency lower bound
        nu: charging/discharging efficiency upper bound
        Ql: Battery capacity (kWh) lower bound
        Qu: Battery capacity (kWh) upper bound
        socl: Minimum SoC value
        socu: Maximum SoC value
        P, n, Q follow uniform distribution.
    """

    def __init__(self, N=10000, seed=2021):
        """
        Parameters
        ----------
        N: int
            Number of EVs
        seed: int
            Random seed
        """
        # --- 1. init ---
        self.N = N
        self.ev_ufparam = dict(Ns=20,
                               Pl=5.0, Pu=7.0,
                               nl=0.88, nu=0.95,
                               Ql=20.0, Qu=30.0,
                               socl=0, socu=1)
        #  --- 1a. uniform distribution parameters range ---
        unit = self.ev_ufparam['socu']/self.ev_ufparam['Ns']
        self.soc_intv = {}
        decimal = 4
        for i in range(self.ev_ufparam['Ns']):
            intv_single = [np.around(i*unit, decimal), np.around((i+1)*unit, decimal)]
            self.soc_intv[i] = intv_single

        #  --- 1b. normal distribution parameters range ---
        ev_pdf_name =               ['soci', 'socd', 'ti1', 'ti2', 'tf1', 'tf2']
        ev_pdf_data = {'mean':     [0.3,    0.8,    -6.5,   17.5,   8.9,    32.9],
                       'var':      [0.05, 0.03, 3.4, 3.4, 3.4, 3.4],
                       'lb':       [0.2, 0.7, 0.0, 5.5, 0.0, 20.9],
                       'ub':       [0.4, 0.9, 5.5, 24.0, 20.9, 24.0],
                       'info':  ['initial SoC', 'demanded SoC',
                                 'start charging time 1', 'start charging time 2',
                                 'finish charging time 1', 'finish charging time 2']}
        ev_pdf = pd.DataFrame(data=ev_pdf_data, index=ev_pdf_name).transpose()
        self.ev_nfparam = ev_pdf.to_dict()

        # --- 1c. generate EV dataset ---
        self.ev_data = pd.DataFrame()
        np.random.seed(seed)

        #  data from uniform distribution
        cols = ['Pc', 'Pd', 'nc', 'nd', 'Q']
        cols_bound = {'Pc':   ['Pl', 'Pu'],
                      'Pd':   ['Pl', 'Pu'],
                      'nc':   ['nl', 'nu'],
                      'nd':   ['nl', 'nu'],
                      'Q':    ['Ql', 'Qu']}
        for col in cols:
            idxl = cols_bound[col][0]
            idxh = cols_bound[col][1]
            self.ev_data[col] = np.random.uniform(
                low=self.ev_ufparam[idxl],
                high=self.ev_ufparam[idxh],
                size=self.N)

        #  data from normal distribution
        # soci, socd
        for col in ev_pdf_name[0:2]:
            self.ev_data[col] = stats.truncnorm(
                            (ev_pdf[col]['lb'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'], 
                            (ev_pdf[col]['ub'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                            loc=ev_pdf[col]['mean'], scale=ev_pdf[col]['var']).rvs(self.N)
        # ti1, ti2, tf1, tf2

        # --- report init info ---
        msg1 = "EV_SSM initialization:\n"
        msg_ev = f"{self.N} EVs, {self.ev_ufparam['Ns']} SoC intervals"
        logger.warning(msg1 + msg_ev)


sse = ev_ssm()
sse.ev_data

EV_SSM initialization:
10000 EVs, 20 SoC intervals


,Pc,Pd,nc,nd,Q,soci,socd
0,6.211957,5.510671,0.906904,0.880400,21.677801,0.305890,0.818166
1,6.466739,6.283294,0.922561,0.918585,29.654122,0.312636,0.801906
2,5.277894,6.471517,0.900035,0.934780,23.550309,0.358089,0.755766
3,5.625346,6.125349,0.909166,0.942975,23.177159,0.281295,0.832347
4,6.994487,6.657985,0.916865,0.927107,22.621299,0.363501,0.807914
...,...,...,...,...,...,...,...
9995,6.069848,6.658940,0.901234,0.912386,26.808354,0.353520,0.842112
9996,6.013977,6.169299,0.939620,0.904331,21.060151,0.300943,0.811256
9997,6.792458,6.332485,0.893216,0.901565,25.709455,0.352536,0.782727
9998,6.648352,6.770357,0.929059,0.933261,29.533473,0.251251,0.770011


In [34]:
ev_nfparam = {
            'mean':     [0.3, 0.8, -6.5, 17.5, 8.9, 32.9],
            'var':      [0.05, 0.03, 3.4, 3.4, 3.4, 3.4],
            'lb':       [0.2, 0.7, 0.0, 5.5, 0.0, 20.9],
            'ub':       [0.4, 0.9, 5.5, 24.0, 20.9, 24.0],
            'info':  ['initial SoC', 'demanded SoC', 'start charging time 1', 'start charging time 2',
                      'finish charging time 1', 'finish charging time 2']}
name = ['soci', 'socd', 'ti1', 'ti2', 'tf1', 'tf2']
ev_pdf = pd.DataFrame(data = ev_nfparam, index =name).transpose()
ev_pdf.to_dict()['soci']['info']

'initial SoC'

In [ ]:
#  rated charging/discharging power, charging/discharging efficiency, battery capacity follow uniform distribution
ev_param = dict(
  N=10000,      # Number of EV
  Ns=20,        # Number of SoC intervals
  Pl = 5.0,     # Rated charging/discharging power (kW) lower bound
  Pu = 7.0,     # Rated charging/discharging power (kW) upper bound
  nl = 0.88,    # charging/discharging efficiency lower bound
  nu = 0.95,    # charging/discharging efficiency upper bound
  Ql = 20.0,    # Battery capacity (kWh) lower bound
  Qu = 30.0,    # Battery capacity (kWh) upper bound
  socl = 0,     # Minimum SoC value
  socu= 1,      # Maximum SoC value
)

#  soc intervals
unit = ev_param['socu']/ev_param['Ns']
intv = {}
decimal = 4
for i in range(ev_param['Ns']):
    intv_single = [np.around(i*unit, decimal), np.around((i+1)*unit, decimal)]
    intv[i] = intv_single
print(f"SoC are seperated into Ns(={ev_param['Ns']}) intervals:")
print(intv)

#  others follow normal distribution
ev_pdf_name = ['soci', 'socd', 'ti1', 'ti2', 'tf1', 'tf2']
ev_pdf_data = {
    'mean':     [0.3, 0.8, -6.5, 17.5, 8.9, 32.9],
    'var':      [0.05, 0.03, 3.4, 3.4, 3.4, 3.4],
    'lb':       [0.2, 0.7, 0.0, 5.5, 0.0, 20.9],
    'ub':       [0.4, 0.9, 5.5, 24.0, 20.9, 24.0],
    'comment':  ['initial SoC', 'demanded SoC', 'start charging time 1', 'start charging time 2',
                 'finish charging time 1', 'finish charging time 2'],
    }
ev_pdf = pd.DataFrame(data = ev_pdf_data, index =ev_pdf_name).transpose()
print("Parameters that follow a normal distribution is stored in ``ev_pdf``:")
ev_pdf

### Generate EV dataset

The generated EV data are stored in a pd.DataFrame ``ev_data``:

| Parameter | Definition                      |
|-----------|---------------------------------|
| Pc/Pd     | charging/discharging rate       |
| nc/nd     | charging/discharging efficiency |
| Q         | capacity                        |
| soci/socd | initial/demand SoC level        |
| ti/tf     | start/finish time               |

In [ ]:
#  store the EV data in a df
ev_data = pd.DataFrame()

np.random.seed(2021)  # set the random seed

#  data from uniform distribution
cols = ['Pc', 'Pd', 'nc', 'nd', 'Q']
cols_bound = {
    'Pc':   ['Pl', 'Pu'],
    'Pd':   ['Pl', 'Pu'],
    'nc':   ['nl', 'nu'],
    'nd':   ['nl', 'nu'],
    'Q':    ['Ql', 'Qu'],
}

for col in cols:
    idxl = cols_bound[col][0]
    idxh = cols_bound[col][1]
    ev_data[col] = np.random.uniform(low=ev_param[idxl], high=ev_param[idxh], size=ev_param['N'])

#  data from normal distribution
for col in ev_pdf_name[0:2]:
    ev_data[col] = stats.truncnorm(
                    (ev_pdf[col]['lb'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'], 
                    (ev_pdf[col]['ub'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                    loc=ev_pdf[col]['mean'], scale=ev_pdf[col]['var']).rvs(ev_param['N'])

#  data from ref1 is confusing, now data from ref2 is used. 02/06/2022

#  data from ref1
#  there are two types of ti/tf, ratio_t1 is used to control the proportion of ti1/tf1

# initialization
ratio_t1 = 0.5
amt1 = int(ev_param['N']*ratio_t1)
amt2 = ev_param['N'] - amt1
col = 'ti1'
ti1 = stats.truncnorm(
                (ev_pdf[col]['lb'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'], 
                (ev_pdf[col]['ub'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                loc=ev_pdf[col]['mean'], scale=ev_pdf[col]['var']).rvs(amt1)
col = 'tf1'
tf1 = stats.truncnorm(
                (ev_pdf[col]['lb'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'], 
                (ev_pdf[col]['ub'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                loc=ev_pdf[col]['mean'], scale=ev_pdf[col]['var']).rvs(amt1)
col = 'ti2'
ti2 = stats.truncnorm(
                (ev_pdf[col]['lb'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'], 
                (ev_pdf[col]['ub'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                loc=ev_pdf[col]['mean'], scale=ev_pdf[col]['var']).rvs(amt2)
col = 'tf2'
tf2 = stats.truncnorm(
                (ev_pdf[col]['lb'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'], 
                (ev_pdf[col]['ub'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                loc=ev_pdf[col]['mean'], scale=ev_pdf[col]['var']).rvs(amt2)

ti = np.concatenate((ti1, ti2))
tf = np.concatenate((tf1, tf2))
np.random.shuffle(ti)
np.random.shuffle(tf)

ev_data['ti'] = ti
ev_data['tf'] = tf

# while loop
ncount = 0
bad_amount = ev_data[ev_data['ti'] > ev_data['tf']].shape[0]
while ncount < 100 and bad_amount > 0:
    ncount += 1
    # print(f"Loop {ncount}")
    amt1 = int(bad_amount * ratio_t1)
    amt2 = bad_amount - amt1

    col = 'ti1'
    ti1 = stats.truncnorm(
                    (ev_pdf[col]['lb'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'], 
                    (ev_pdf[col]['ub'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                    loc=ev_pdf[col]['mean'], scale=ev_pdf[col]['var']).rvs(amt1)
    col = 'tf1'
    tf1 = stats.truncnorm(
                    (ev_pdf[col]['lb'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'], 
                    (ev_pdf[col]['ub'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                    loc=ev_pdf[col]['mean'], scale=ev_pdf[col]['var']).rvs(amt1)
    col = 'ti2'
    ti2 = stats.truncnorm(
                    (ev_pdf[col]['lb'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'], 
                    (ev_pdf[col]['ub'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                    loc=ev_pdf[col]['mean'], scale=ev_pdf[col]['var']).rvs(amt2)
    col = 'tf2'
    tf2 = stats.truncnorm(
                    (ev_pdf[col]['lb'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'], 
                    (ev_pdf[col]['ub'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                    loc=ev_pdf[col]['mean'], scale=ev_pdf[col]['var']).rvs(amt2)
    
    ti = np.concatenate((ti1, ti2))
    tf = np.concatenate((tf1, tf2))
    np.random.shuffle(ti)
    np.random.shuffle(tf)

    old_idx = ev_data['ti'][ev_data['ti'] > ev_data['tf']].index
    ev_data['ti'][old_idx] = ti
    ev_data['tf'][old_idx] = tf

    bad_amount = ev_data[ev_data['ti'] > ev_data['tf']].shape[0]

#  data consistance check:
#  1) soci <= socd; 2) ti <= tf
cond1 = ev_data[ev_data['soci'] > ev_data['socd']].shape[0] == 0
cond2 = ev_data[ev_data['ti'] > ev_data['tf']].shape[0] == 0
if cond1 and cond2:
    print("Consistency check passed.")

#  sample data
ev_data.head()

### Examine EV data

The data looks unreasonable for now, may need to update the data set. But we can continue with it for now.

In [ ]:
plt.hist(ev_data["ti"])
plt.hist(ev_data["tf"])
plt.show()

plt.hist(ev_data["Q"])

## SSM Matrix

In [ ]:
ev_pdf_name[0:2]

## Simulation

Issues: how to integerate the SSM when using ADNES?

flow_chart:
```{python}
prep grid data:
ADNES: topology,  gen. limits, ramp. limits, line limits,
Outside: gen. cost, ramp. cost,

for $t_{OPF}$ in T (interval: 5min; total: 1h; [n=12]):
    aggregate EV data (from SSM), generate $PR_{e,i,u,t}$
    Do OPF, generate $PG_{i, t}$, $PR_{g, i, u, t}$, $PR_{g, i, d, t}$

    for t in $t_{OPF}$ (interval: 4s; total: 5min; [n=75]):
        Update data into dynamic system:
            # Note, constant power model should be used in TDS.
            # Use TimeSeries as the load. 
            power change: TGOV1.paux0
            load change: 

        Run TDS: generate SFR mileage
```